# Conversion Rate Queries

In [ ]:
import os
from pathlib import Path

from dotenv import find_dotenv, load_dotenv
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL

In [ ]:
PROJ_ROOT = Path().resolve().parents[3]
env_file_dir = PROJ_ROOT / '.env'
_ = load_dotenv(env_file_dir, verbose=True)

## About

Develop **conversion rate** queries per product and overall.

### Notes

1. This notebook supports <kbd>Run</kbd> > <kbd>Run All Cells</kbd>.

## User Inputs

In [ ]:
#

In [ ]:
engine = create_engine(
    URL(
        drivername="driver",
        account=os.getenv("UPLIMIT_SNOWFLAKE_ACCOUNT"),
        user=os.getenv("UPLIMIT_SNOWFLAKE_USER"),
        password=os.getenv("UPLIMIT_SNOWFLAKE_PASS"),
        warehouse=os.getenv("UPLIMIT_SNOWFLAKE_WAREHOUSE"),
        role=os.getenv("UPLIMIT_SNOWFLAKE_ROLE"),
        database=os.getenv("UPLIMIT_SNOWFLAKE_DB_NAME"),
        schema=os.getenv("UPLIMIT_SNOWFLAKE_SCHEMA"),
        timezone='US/Eastern'
    )
)

## Connect

Load Jupyter SQL extension

In [ ]:
%load_ext sql

Set the maximum number of rows to be displayed to `None` (shows all rows)

In [ ]:
%config SqlMagic.displaylimit = None

Connect to DuckDB database

In [ ]:
%sql engine --alias connection

## Exploratory Data Analysis

Show the different types of events

In [ ]:
%%sql
SELECT DISTINCT(event_type) AS event_type
FROM stg_postgres_events

Show the first few rows of the events table

In [ ]:
%%sql
SELECT *
FROM stg_postgres_events
LIMIT 4

Show chronolgically ordered events in 15 sessions in which there was no checkout or package shipment

In [ ]:
%%sql
WITH t1 AS (
    SELECT * EXCLUDE(event_id),
           ROW_NUMBER() OVER(
               PARTITION BY session_id ORDER BY user_id, created_at DESC
           ) AS rn
    FROM stg_postgres_events
),
t2 AS (
    SELECT session_id
    FROM t1
    -- get sessions in which last event is not shipping or checkout
    WHERE (rn = 1 AND event_type NOT IN ('checkout', 'package_shipped'))
    ORDER BY session_id
    LIMIT 15
),
sessions_without_purchase AS (
    SELECT * EXCLUDE(event_id)
    FROM stg_postgres_events
    -- user INNER JOIN to only get sessions in which order was not placed
    INNER JOIN t2 USING (session_id)
    ORDER BY session_id, user_id, created_at
)
SELECT *
FROM sessions_without_purchase

A single session is associated with a single user. There are no multi-user sessions

In [ ]:
%%sql
WITH multi_user_sessions AS (
    SELECT session_id,
           COUNT(DISTINCT(user_id)) AS num_users
    FROM stg_postgres_events
    GROUP BY ALL
    HAVING num_users > 1
)
SELECT COUNT(*) AS num_multi_user_sessions
FROM multi_user_sessions

A single user can have multiple sessions

In [ ]:
%%sql
WITH sessions_per_user AS (
    SELECT user_id,
           COUNT(DISTINCT(session_id)) AS num_sessions
    FROM stg_postgres_events
    GROUP BY ALL
),
single_multi_session_users AS (
    SELECT *
    FROM (
        SELECT '1' AS num_sessions,
               COUNT(*) AS num_users,
               False AS is_multi_session_user
        FROM sessions_per_user
        WHERE num_sessions = 1
        GROUP BY ALL
    )
    UNION ALL
    SELECT '1+' AS num_sessions,
           COUNT(*) AS num_users,
           True AS is_multi_session_user
    FROM sessions_per_user
    WHERE num_sessions > 1
)
SELECT *
FROM single_multi_session_users

As seen from the total of the `num_users` column in the above query result, all users in the `users` table do not have an entry in the `events` table. This is verified below

In [ ]:
%%sql
WITH num_users_from_events AS (
    SELECT COUNT(DISTINCT(user_id)) AS num_users_events,
           1 AS row_num
    FROM stg_postgres_events
),
num_users_overall AS (
    SELECT COUNT(DISTINCT(user_id)) AS num_users,
           1 AS row_num
    FROM stg_postgres_users
)
SELECT *
FROM num_users_from_events
INNER JOIN num_users_overall USING (row_num)

## Product Queries - Conversion Rate

### By Product

#### Using `staging` models

In [ ]:
%%sql
-- ####### START INTERMEDIATE MODEL #######
/* get product events for sessions that did not end in a purchase */
WITH products_non_purchase_sessions AS (
    SELECT session_id,
           user_id,
           event_id,
           event_type,
           product_id,
           created_at,
           0 AS is_purchased
    FROM stg_postgres_events
    -- get sessions in which the last event does not indicate a purchase
    QUALIFY (
        LAST_VALUE(event_type)
        OVER(PARTITION BY session_id ORDER BY session_id, created_at)
    ) IN ('page_view', 'add_to_cart')
),
/* get the session ID for sessions ending in a purchase */
sessions_with_purchase AS (
    SELECT DISTINCT(session_id) AS session_id
    FROM stg_postgres_events
    -- get sessions in which the last event indicates a purchase
    QUALIFY (
        LAST_VALUE(event_type)
        OVER(PARTITION BY session_id ORDER BY session_id, created_at)
    ) IN ('checkout', 'package_shipped')
),
/* get product events for sessions that did convert to a purchase */
products_purchase_sessions AS (
    SELECT s.session_id,
           s.user_id,
           s.event_id,
           s.event_type,
           s.product_id,
           s.created_at,
           1 AS is_purchased
    FROM stg_postgres_events s
    -- user INNER JOIN to only get sessions ending in a purchase
    INNER JOIN sessions_with_purchase sp USING (session_id)
    -- get events showing the ID of the purchased product
    WHERE product_id IS NOT NULL
    ORDER BY session_id, created_at
),
/* count sessions not ending in a purchase in which product page was viewed */
product_non_purchase_page_views AS (
    SELECT product_id,
           COUNT(DISTINCT(session_id)) AS num_non_purchase_page_view_sessions
    FROM products_non_purchase_sessions
    WHERE event_type = 'page_view'
    GROUP BY product_id
),
/* count sessions ending in a purchase in which product page was viewed */
product_purchase_page_views AS (
    SELECT product_id,
           COUNT(DISTINCT(session_id)) AS num_purchase_page_view_sessions
    FROM products_purchase_sessions
    WHERE event_type = 'page_view'
    GROUP BY product_id
),
/* count sessions ending in a purchase */
product_purchases AS (
    SELECT product_id,
           COUNT(DISTINCT(session_id)) AS num_purchase_sessions
    FROM products_purchase_sessions
    -- get add-to-cart events since only products in a cart can be purchased
    -- (exclude products with a page_view that are not followed by add-to-cart
    -- since these cannot be purchased)
    WHERE event_type = 'add_to_cart'
    GROUP BY product_id
),
/* join three types of session counts */
product_session_totals AS (
    SELECT pp.product_id,
           npv.num_non_purchase_page_view_sessions,
           ppv.num_purchase_page_view_sessions,
           pp.num_purchase_sessions
    FROM product_non_purchase_page_views npv
    INNER JOIN product_purchases pp USING (product_id)
    INNER JOIN product_purchase_page_views ppv USING (product_id)
),
-- ####### END INTERMEDIATE MODEL #######
products AS (
    SELECT product_id,
           name AS product_name
    FROM stg_postgres_products
),
/* get conversion rate */
product_conversion_rates AS (
    SELECT product_id,
           (
               num_non_purchase_page_view_sessions
               +num_purchase_page_view_sessions
           ) AS total_num_page_view_sessions,
           num_purchase_sessions,
           (
               100*num_purchase_sessions/total_num_page_view_sessions
           ) AS conversion_rate
    FROM product_session_totals
),
/* get first and last event timestamp */
product_event_timestamp_bounds AS (
    SELECT product_id,
           MIN(created_at) AS first_event,
           MAX(created_at) AS last_event
    FROM stg_postgres_events
    GROUP BY product_id
),
/* combine conversion rates and event timestamp bounds */
product_conversion_rates_timestamp_bounds AS (
    SELECT pn.product_name,
           c.total_num_page_view_sessions,
           c.num_purchase_sessions,
           c.conversion_rate,
           b.first_event,
           b.last_event
    FROM product_conversion_rates c
    INNER JOIN product_event_timestamp_bounds b USING (product_id)
    INNER JOIN products pn USING (product_id)
    ORDER BY conversion_rate DESC
)
SELECT *
FROM product_conversion_rates_timestamp_bounds

### Definitions

1. Number of sessions with product (page) views and without a purchase
   - `num_non_purchase_page_view_sessions`
   - number of sessions ending without a purchase in which a product page was viewed
2. Number of sessions with product (page) views and with a purchase
   - `num_purchase_page_view_sessions`
   - number of sessions ending with a purchase in which a product page was viewed
3. Total sessions with product page views
   - `total_num_page_view_sessions`
   - sum of 1. and 2. above
4. Number of sessions with product purchases
   - `num_purchase_sessions`
   - number of sessions ending with a purchase
     - since only one purchase is allowed per session, this is the same as `num_purchases` (number of purcases)
5. Product conversion rate
   - `conversion_rate`
   - 100 X number of sessions with product purchase / Total sessions with product page views

#### Using `intermediate` models

In [ ]:
%%sql
WITH products AS (
    SELECT product_id,
           name AS product_name
    FROM stg_postgres_products
),
/* get conversion rate */
product_conversion_rates AS (
    SELECT product_id,
           (
               num_non_purchase_page_view_sessions
               +num_purchase_page_view_sessions
           ) AS total_num_page_view_sessions,
           num_purchases AS num_purchase_sessions,
           (
               100*num_purchase_sessions/total_num_page_view_sessions
           ) AS conversion_rate
    FROM int_events_sessions_aggregated_to_product
),
/* get first and last event timestamp */
product_event_timestamp_bounds AS (
    SELECT product_id,
           MIN(created_at) AS first_event,
           MAX(created_at) AS last_event
    FROM stg_postgres_events
    GROUP BY product_id
),
/* combine conversion rates and event timestamp bounds */
product_conversion_rates_timestamp_bounds AS (
    SELECT p.product_name,
           c.total_num_page_view_sessions,
           c.num_purchase_sessions,
           c.conversion_rate,
           b.first_event,
           b.last_event
    FROM product_conversion_rates c
    INNER JOIN product_event_timestamp_bounds b USING (product_id)
    INNER JOIN products p USING (product_id)
    ORDER BY conversion_rate DESC
)
SELECT *
FROM product_conversion_rates_timestamp_bounds

### Overall

Get conversion rate per product using `intermediate` models

In [ ]:
%%sql
WITH products AS (
    SELECT 1 AS product_id,
           'all' AS product_name
),
/* get conversion rate */
overall_conversion_rates AS (
    SELECT 1 AS product_id,
           SUM(
               num_non_purchase_page_view_sessions
               +num_purchase_page_view_sessions
           ) AS total_num_page_view_sessions,
           SUM(num_purchases) AS num_purchase_sessions,
           (
               100*num_purchase_sessions/total_num_page_view_sessions
           ) AS conversion_rate
    FROM int_events_sessions_aggregated_to_product
),
/* get first and last event timestamp */
overall_event_timestamp_bounds AS (
    SELECT 1 AS product_id,
           MIN(created_at) AS first_event,
           MAX(created_at) AS last_event
    FROM stg_postgres_events
),
/* combine conversion rates and event timestamp bounds */
overall_conversion_rates_timestamp_bounds AS (
    SELECT p.product_name,
           c.total_num_page_view_sessions,
           c.num_purchase_sessions,
           c.conversion_rate,
           b.first_event,
           b.last_event
    FROM overall_conversion_rates c
    INNER JOIN overall_event_timestamp_bounds b USING (product_id)
    INNER JOIN products p USING (product_id)
)
SELECT *
FROM overall_conversion_rates_timestamp_bounds

## Disconnect

Close connection

In [ ]:
%sql --close connection